# GroupByRule: deduplicate data using fuzzy and deterministic matching rules

🚧 under construction 🚧

**GroupByRule** is a Python package for data cleaning and deduplication. It integrates with [pandas](https://pandas.pydata.org/)' [`groupby`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html) function to not only group dataframe rows by a given identifier, but also groups rows based on logical rules and partial matching. In other words, it provides tools for deterministic record linkage and entity resolution in structured databases. It can also be used for *blocking*, a form of filtering used to speed-up more complex entity resolution algorithms. See the references below to learn more about these topics.

One of the main goal of **GroupByRule** is to be user-friendly. Matching rules and clustering algorithms are composable and the performance of algorithms can be readily evaluated given training data. The package is built on top of [pandas](https://pandas.pydata.org) for data manipulation and on [igraph](https://igraph.org/python/) for graph clustering and related computations.

## Installation

Install from github using the following command:

     pip install git+https://github.com/OlivierBinette/groupbyrule.git

## Examples

### Rule-Based Linkage

Consider the `RLdata500` dataset from the [RecordLinkage R package](https://www.google.com/search?channel=fs&client=ubuntu&q=recordlinkage+r+package).

In [3]:
from groupbyrule.data import RLdata500

df = RLdata500()
df

,fname_c1,fname_c2,lname_c1,lname_c2,by,bm,bd,identity
1,CARSTEN,NaN,MEIER,NaN,1949,7,22,34
2,GERD,NaN,BAUER,NaN,1968,7,27,51
3,ROBERT,NaN,HARTMANN,NaN,1930,4,30,115
4,STEFAN,NaN,WOLFF,NaN,1957,9,2,189
5,RALF,NaN,KRUEGER,NaN,1966,1,13,72
...,...,...,...,...,...,...,...,...
496,GABRIHELE,NaN,BECKER,NaN,1990,3,27,413
497,SABINE,NaN,SCHNEIDER,NaN,1953,5,20,378
498,MARIA,NaN,SCHNEIDER,NaN,1981,8,8,399
499,INGE,NaN,SCHREIBER,NaN,1967,12,13,315


We deduplicate this dataset by linking records which match either on both first name (`fname_c1`) and last name (`lname_c1`), on both first name and birth day (`bd`), or on both last name and birth day. Linkage transitivity is resolved, by default, by considering connected components of the resulting graph. Precision and recall are computed from the ground truth membership vector `identity_RLdata500`.

In [7]:
from groupbyrule import Any, Match, precision_recall

# Specify linkage rule
rule = Any(Match("fname_c1", "lname_c1"),
           Match("fname_c1", "bd"),
           Match("lname_c1", "bd"))

# Apply the rule to a dataset
rule.fit(df)

# Evaluate performance by computing precision and recall
precision_recall(rule.groups, df.identity)

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 18,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 1,
 41,
 42,
 43,
 44,
 45,
 46,
 18,
 37,
 47,
 48,
 49,
 50,
 51,
 52,
 18,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 18,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 18,
 75,
 67,
 76,
 61,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 36,
 94,
 55,
 24,
 95,
 96,
 97,
 32,
 98,
 99,
 100,
 101,
 98,
 77,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 16,
 109,
 21,
 110,
 111,
 112,
 113,
 114,
 71,
 115,
 116,
 23,
 117,
 96,
 118,
 84,
 119,
 120,
 121,
 36,
 37,
 69,
 112,
 18,
 122,
 123,
 124,
 125,
 18,
 126,
 127,
 18,
 128,
 129,
 96,
 99,
 20,
 130,
 131,
 132,
 104,
 133,
 119,
 134,
 135,
 97,
 136,
 137,
 100,
 138,
 55,
 139,
 140,
 141,
 142,
 143,
 144,
 45,
 18,
 145,
 146,
 18,
 76,
 147,
 148,
 62,
 67,
 14


This is not the best way to deduplicate this dataset, but the above showcases the composability of matching rules. The specific rules themselves (exact matching, similarity-based string matching, and different clustering algorithms) can be customized as needed. A more complete overview is available [here]() 🚧.

A better way to deduplicate this data is to link all pairs of records which agree on all but at most one attribute. This is done below.

In [ ]:
from groupbyrule import AllButK

# Link records agreeing on all but at most k=1 of the specified attributes
rule = AllButK("fname_c1", "lname_c1", "bd", "bm", "by", k=1)

# Apply the rule to a dataset
rule.fit(df)

# Evaluate performance by computing precision and recall
precision_recall(rule.groups, df.identity)

(1.0, 0.92)

### Postprocessing

Following record linkage, records can be processed using pandas's groupby and aggregation functions. Below, we only keep the first non-NA attribute value for each record cluster. This is a simple way to obtain a deduplicated dataset.

In [ ]:
df.groupby(rule.groups).first()

,fname_c1,fname_c2,lname_c1,lname_c2,by,bm,bd,identity
0,CARSTEN,None,MEIER,None,1949,7,22,34
1,GERD,None,BAUER,None,1968,7,27,51
2,ROBERT,None,HARTMANN,None,1930,4,30,115
3,STEFAN,None,WOLFF,None,1957,9,2,189
4,RALF,None,KRUEGER,None,1966,1,13,72
...,...,...,...,...,...,...,...,...
449,BRITTA,None,KOEHLER,None,2001,1,12,424
450,SABINE,None,SCHNEIDER,None,1953,5,20,378
451,MARIA,None,SCHNEIDER,None,1981,8,8,399
452,INGE,None,SCHREIBER,None,1967,12,13,315


In [ ]:
from groupbyrule.data import ABSEmployee

df = ABSEmployee()

df

,RECID,FILEID,ENTID,SA1,MB,BDAY,BYEAR,SEX,INDUSTRY,CASUAL,FULLTIME,HOURS,PAYRATE,AWE
0,A000001,A,1,10929,1092903.0,168.0,1954,2,4.0,0,0,17,35.00,595.00
1,A000002,A,2,10981,1098109.0,26.0,1998,1,1.0,0,1,40,40.00,1600.00
2,A000006,A,6,10768,1076809.0,168.0,1990,2,2.0,0,1,40,43.20,1728.00
3,A000009,A,9,10399,1039905.0,344.0,1997,1,1.0,0,1,42,41.00,1722.00
4,A000012,A,12,10616,1061607.0,190.0,1954,2,2.0,0,1,40,45.00,1800.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
659995,C399994,C,399994,10381,1038105.0,30.0,1964,2,5.0,0,0,16,37.10,593.60
659996,C399996,C,399996,10508,1050807.0,150.0,1992,1,1.0,0,1,41,46.50,1906.50
659997,C399997,C,399997,10969,1096907.0,78.0,1988,2,3.0,1,0,18,47.52,855.36
659998,C399998,C,399998,10096,1009601.0,67.0,1968,1,5.0,1,0,6,44.94,269.64


In [ ]:
#%load_ext line_profiler
rule = AllButK("BDAY", "BYEAR", "SEX", "INDUSTRY", "CASUAL", "FULLTIME", k=1)

df = df.iloc[1:10000, :]

rule.fit(df)

NameError: name 'AllButK' is not defined

In [ ]:
import numpy as np
precision_recall(rule.groups, df.ENTID)

AttributeError: 'DataFrame' object has no attribute 'ENTID'


### Similarity-Based Linkage Rules

🚧

### Supervised Approaches and Learning Rules

🚧

### Clustering Algorithms

🚧

### Performance Evaluation

🚧

## References

🚧
